In [25]:
import pandas as pd

In [26]:
Data = pd.read_csv(r"D:\ALGORITMO RECLUTADOR\Todo el proyecto\Data Posición\Delanteros.csv")
pd.set_option('display.max_columns', None)

In [27]:
print(Data.columns)

Index(['Name', 'Goals', 'Big chances missed', 'Successful dribbles %',
       'Total shots', 'Headed goals', 'Set piece conversion %',
       'Interceptions', 'Big chances created', 'Assists', 'Accurate passes %',
       'Aerial duels won %', 'Total duels won %', 'Was fouled', 'Appearances',
       'Age Range', 'Position', 'League', 'Level'],
      dtype='object')


In [28]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

Columna_habilidades = Data.columns.drop(["Name", "League"])

features = Data[Columna_habilidades]

# Crea un objeto StandardScaler
scaler = StandardScaler()

# Ajusta el escalador a tus datos y luego transforma tus datos
features_scaled = scaler.fit_transform(features)

# Inicializa el modelo NearestNeighbors
model = NearestNeighbors(n_neighbors=6)  # Buscamos 6 vecinos porque uno de ellos será el jugador mismo

# Ajusta el modelo a tus datos
model.fit(features_scaled)


NearestNeighbors(n_neighbors=6)

In [29]:
Data.columns

Index(['Name', 'Goals', 'Big chances missed', 'Successful dribbles %',
       'Total shots', 'Headed goals', 'Set piece conversion %',
       'Interceptions', 'Big chances created', 'Assists', 'Accurate passes %',
       'Aerial duels won %', 'Total duels won %', 'Was fouled', 'Appearances',
       'Age Range', 'Position', 'League', 'Level'],
      dtype='object')

In [30]:
caracteristicas_grafico = ['Goals', 'Big chances missed', 'Successful dribbles %', 'Total shots', 'Headed goals', 'Set piece conversion %', 'Assists']

In [48]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import unicodedata
import plotly.graph_objects as go

# ... (Asumiendo que ya tienes tu DataFrame 'Data' y otras configuraciones necesarias)

# Función para normalizar nombres (ignorar mayúsculas y acentos)
def normalizar_nombre(nombre):
    nombre = nombre.lower()
    nombre = unicodedata.normalize('NFD', nombre)
    nombre = nombre.encode('ascii', 'ignore').decode("utf-8")
    return nombre

def buscar_jugadores_similares(nombre, caracteristicas_grafico, Goals_max, Goals_media, Goals_min, Successful_max, Successful_media, Successful_min, shots_max, shots_media, shots_min, missed_max, missed_media, missed_min):
    # Normaliza el nombre que buscas
    nombre_normalizado = normalizar_nombre(nombre)

    # Aplica la normalización a los nombres en el conjunto de datos
    Data['Name_Normalized'] = Data['Name'].apply(normalizar_nombre)

    # Busca el jugador con el nombre normalizado
    jugador = Data[Data['Name_Normalized'] == nombre_normalizado]

    if len(jugador) == 0:
        return 'Jugador no encontrado'
    else:
        jugador_scaled = scaler.transform(jugador[features.columns])
        distancias, indices = model.kneighbors(jugador_scaled)
        jugadores_similares = Data.iloc[indices[0]]  # Excluimos el primer resultado porque será el jugador mismo

        # Escala todas las características de los jugadores similares
        jugadores_similares_scaled = scaler.transform(jugadores_similares[features.columns])

        # Define una lista de colores
        colores = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black']

        # Crea un gráfico de radar para cada jugador similar
        fig = go.Figure()
        for i, row in enumerate(jugadores_similares_scaled):
            color = colores[i % len(colores)]  # Elige un color de la lista
            fig.add_trace(go.Scatterpolar(
                r=row[features.columns.isin(caracteristicas_grafico)],
                theta=caracteristicas_grafico,
                fill='toself',
                name=jugadores_similares.iloc[i]['Name'],
                line=dict(color=color)
            ))

        fig.update_layout(
            autosize=False,
            width=800,
            height=800,
            polar=dict(
                radialaxis=dict(
                    visible=True,
                    range=[-2, 10]  # Ajusta este rango según tus datos
                )),
            showlegend=True
        )
        fig.show()
        
        # Configura una cuadrícula de subplots con 1 fila y 4 columnas
        fig, axs = plt.subplots(1, 4, figsize=(20, 5))  # Ajusta el tamaño según sea necesario

        # Gráfico de dispersión para la media-Goals
        for i, name in enumerate(jugadores_similares['Name']):
            axs[0].scatter(i, jugadores_similares.loc[jugadores_similares['Name'] == name, 'Goals'], color=colores[i % len(colores)], marker='o')
        for line in [Goals_max, Goals_media, Goals_min]:
            axs[0].axhline(y=line, color='r', linestyle='--')
        axs[0].set_title('Media de Goals de los jugadores similares')
        axs[0].set_xticks(range(len(jugadores_similares)))
        axs[0].set_xticklabels(jugadores_similares['Name'], rotation=360)
        axs[0].set_xlabel('Jugadores')
        axs[0].set_ylabel('Goals')

        # Gráfico de dispersión para la media-Successful
        for i, name in enumerate(jugadores_similares['Name']):
            axs[1].scatter(i, jugadores_similares.loc[jugadores_similares['Name'] == name, 'Successful dribbles %'], color=colores[i % len(colores)], marker='o')
        for line in [Successful_max, Successful_media, Successful_min]:
            axs[1].axhline(y=line, color='r', linestyle='--')
        axs[1].set_title('Media de Successful')
        axs[1].set_xticks(range(len(jugadores_similares)))
        axs[1].set_xticklabels(jugadores_similares['Name'], rotation=360)
        axs[1].set_xlabel('Jugadores')
        axs[1].set_ylabel('Successful')

        # Gráfico de dispersión para la media-shots
        for i, name in enumerate(jugadores_similares['Name']):
            axs[2].scatter(i, jugadores_similares.loc[jugadores_similares['Name'] == name, 'Total shots'], color=colores[i % len(colores)], marker='o')
        for line in [shots_max, shots_media, shots_min]:
            axs[2].axhline(y=line, color='r', linestyle='--')
        axs[2].set_title('Media de shots de los jugadores similares')
        axs[2].set_xticks(range(len(jugadores_similares)))
        axs[2].set_xticklabels(jugadores_similares['Name'], rotation=360)
        axs[2].set_xlabel('Jugadores')
        axs[2].set_ylabel('shots')

        # Gráfico de dispersión para la media-missed
        for i, name in enumerate(jugadores_similares['Name']):
            axs[3].scatter(i, jugadores_similares.loc[jugadores_similares['Name'] == name, 'Big chances missed'], color=colores[i % len(colores)], marker='o')
        for line in [missed_max, missed_media, missed_min]:
            axs[3].axhline(y=line, color='r', linestyle='--')
        axs[3].set_title('Media de missed de los jugadores similares')
        axs[3].set_xticks(range(len(jugadores_similares)))
        axs[3].set_xticklabels(jugadores_similares['Name'], rotation=360)
        axs[3].set_xlabel('Jugadores')
        axs[3].set_ylabel('missed')

        # Ajusta el layout para que no haya superposición de elementos
        plt.tight_layout()
        plt.show()

        return jugadores_similares




In [50]:
buscar_jugadores_similares('domenico Bèrardi', caracteristicas_grafico)

TypeError: buscar_jugadores_similares() missing 12 required positional arguments: 'Goals_max', 'Goals_media', 'Goals_min', 'Successful_max', 'Successful_media', 'Successful_min', 'shots_max', 'shots_media', 'shots_min', 'missed_max', 'missed_media', and 'missed_min'